# Linear Regression: Demo

In [ ]:
# Start with importing the usual stuff

import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import statsmodels.api as sm
from sklearn.model_selection import train_test_split 

pd.options.mode.copy_on_write = True

In [ ]:
# This contains some functions written by the authors of the textbook (ISLP)
# we'll be adapting most of the this demo from their material

from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)

## Set up data

In [ ]:
# First we'll load the "Advertising" data

Advertising = pd.read_csv('../../data/Advertising.csv')
#Advertising = pd.read_csv('Advertising.csv')
Advertising

In [ ]:
# Identify predictors and target

X = Advertising[['TV', 'radio', 'newspaper']]
y = Advertising['sales']

In [ ]:
# Create a column for the intercept in the features matrix

X['intercept'] = np.ones(Advertising.shape[0])
X

In [ ]:
# Create training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=314,
                                                    test_size=0.25,
                                                    shuffle=True) 
Train = pd.merge_ordered(X_train,y_train,left_on=X_train.index,right_on=y_train.index).drop(columns=['key_0'])
Test = pd.merge_ordered(X_test,y_test,left_on=X_test.index,right_on=y_test.index).drop(columns=['key_0'])

In [ ]:
# Spot check the train and test sets

print(X_train.head()) 
print(X_train.shape)
print()
print(y_train.head()) 
print(y_train.shape)
print() 
print(X_test.head()) 
print(X_test.shape)
print() 
print(y_test.head())
print(y_test.shape)
print() 
print(Train.head())
print(Train.shape)
print() 
print(Test.head())
print(Test.shape)

In [ ]:
# Look at possible values and spot check for missing entries

print('Sales')
print(np.unique(Train['sales']))
print('TV')
print(np.unique(Train['TV']))
print('Radio')
print(np.unique(Train['radio']))
print('Newspaper')
print(np.unique(Train['newspaper']))

## Simple Linear Regression, Example 1

In [ ]:
# Build a linear model where TV predicts sales
# Take note of the p-values

model_TV = sm.OLS(y_train, X_train[['intercept','TV']])
results_TV = model_TV.fit()
summarize(results_TV)

In [ ]:
# We can also assess the overall fit of the model

print('R^2 on train:',results_TV.rsquared)

In [ ]:
# Create helper functions for computing the mean squared error

def predict(X, model):
    # the built-in get_prediction tool returns an array, so we need to convert to a dataframe
    predictions_df = pd.DataFrame(model.get_prediction(X).predicted, columns=['y_hat'], index=X.index)
    return predictions_df['y_hat']

def mse(y, y_hat):
    # calculate the residual error for each individual record
    resid = y - y_hat
    # square the residual (hence "squared error")
    sq_resid = resid**2
    # calculate the sum of squared errors
    SSR = sum(sq_resid)
    # divide by the number of records to get the mean squared error
    MSE = SSR / y.shape[0]
    return MSE

In [ ]:
# Let's compute the MSE on the training and test sets

predictions_TV_train = predict(X_train[['intercept', 'TV']], results_TV)
print('mse train:',mse(y_train, predictions_TV_train))
predictions_TV_test = predict(X_test[['intercept', 'TV']], results_TV)
print('mse test:',mse(y_test, predictions_TV_test))

In [ ]:
# Define a function to draw a line given coefficients [credit to Hastie & Tibshirani]

def abline(ax, b, m, *args, **kwargs):
    "Add a line with slope m and intercept b to ax"
    xlim = ax.get_xlim()
    ylim = [m * xlim[0] + b, m * xlim[1] + b]
    ax.plot(xlim, ylim, *args, **kwargs)

In [ ]:
# Plot TV vs sales on training set

ax = Train.plot.scatter('TV', 'sales')
ax.set_title("Plot of TV vs Sales (Train)")
abline(ax,
       results_TV.params[0],
       results_TV.params[1],
       'r--',
       linewidth=3)

In [ ]:
# Plot TV vs sales on test set

ax = Test.plot.scatter('TV', 'sales')
ax.set_title("Plot of TV vs Sales (Test)")
abline(ax,
       results_TV.params[0],
       results_TV.params[1],
       'g--',
       linewidth=3)

In [ ]:
# Plot residual error for train set

ax = subplots(figsize=(8,8))[1]
ax.scatter(predictions_TV_train, y_train-predictions_TV_train)
ax.set_xlabel('Fitted value')
ax.set_ylabel('Residual')
ax.axhline(0, c='k', ls='--');

In [ ]:
# Plot residual error for test set

ax = subplots(figsize=(8,8))[1]
ax.scatter(predictions_TV_test, y_test-predictions_TV_test)
ax.set_xlabel('Fitted value')
ax.set_ylabel('Residual')
ax.axhline(0, c='k', ls='--');

## Simple Linear Regression, Example 2

In [ ]:
# Build a linear model where radio predicts sales

model_radio = sm.OLS(y_train, X_train[['intercept','radio']])
results_radio = model_radio.fit()
summarize(results_radio)

In [ ]:
# R^2 for radio model
print('R^2 on train:',results_radio.rsquared)

In [ ]:
# MSE for radio model

predictions_radio_train = predict(X_train[['intercept', 'radio']], results_radio)
print('mse train:',mse(y_train, predictions_radio_train))
predictions_radio_test = predict(X_test[['intercept', 'radio']], results_radio)
print('mse test:',mse(y_test, predictions_radio_test))

In [ ]:
# Plot radio vs sales on training set

ax = Train.plot.scatter('radio', 'sales')
ax.set_title("Plot of Radio vs Sales (Train)")
abline(ax,
       results_radio.params[0],
       results_radio.params[1],
       'r--',
       linewidth=3)

In [ ]:
# Plot radio vs sales on test set

ax = Test.plot.scatter('radio', 'sales')
ax.set_title("Plot of Radio vs Sales (Test)")
abline(ax,
       results_radio.params[0],
       results_radio.params[1],
       'g--',
       linewidth=3)

In [ ]:
# Plot residual error for training set

ax = subplots(figsize=(8,8))[1]
ax.scatter(predictions_radio_train, y_train-predictions_radio_train)
ax.set_xlabel('Fitted value')
ax.set_ylabel('Residual')
ax.axhline(0, c='k', ls='--');

In [ ]:
# Plot residual error for test set

ax = subplots(figsize=(8,8))[1]
ax.scatter(predictions_radio_test, y_test-predictions_radio_test)
ax.set_xlabel('Fitted value')
ax.set_ylabel('Residual')
ax.axhline(0, c='k', ls='--');

## Simple Linear Regression, Example 3

In [ ]:
# Build a linear model where newspaper predicts sales

model_newspaper = sm.OLS(y_train, X_train[['intercept','newspaper']])
results_newspaper = model_newspaper.fit()
summarize(results_newspaper)

In [ ]:
# R^2 for newspaper model

print('R^2 on train:',results_newspaper.rsquared)

In [ ]:
# MSE for newspaper model

predictions_newspaper_train = predict(X_train[['intercept','newspaper']], results_newspaper)
print('mse train:',mse(y_train, predictions_newspaper_train))
predictions_newspaper_test = predict(X_test[['intercept','newspaper']], results_newspaper)
print('mse test:',mse(y_test, predictions_newspaper_test))

In [ ]:
# Plot newspaper vs sales

ax = Train.plot.scatter('newspaper', 'sales')
abline(ax,
       results_newspaper.params[0],
       results_newspaper.params[1],
       'r--',
       linewidth=3)

## Multiple Linear Regression

In [ ]:
# Let's look at the correlations in the set

Train.corr()

In [ ]:
import seaborn as sns

# Create a heatmap with the correlation matrix
sns.heatmap(Train.corr(), 
            annot=True,            # Show correlation coefficients
            fmt=".2f",            # Format the annotations
            cmap='coolwarm',      # Color map
            square=True,          # Square cells
            cbar_kws={"shrink": .8})  # Color bar size

In [ ]:
# Build a linear model with both TV and radio as predictors of sales

model_both = sm.OLS(y_train, X_train[['intercept','TV','radio']])
results_both = model_both.fit()
print(summarize(results_both))

In [ ]:
# Let's remind ourselves what the parameters were when we fit TV and radio separately

print(summarize(results_TV))
print('')
print(summarize(results_radio))

In [ ]:
# R^2 

print('R^2 on train:',results_both.rsquared)

In [ ]:
# MSE

predictions_both_train = predict(X_train[['intercept','TV','radio']], results_both)
print('mse train:',mse(y_train, predictions_both_train))
predictions_both_test = predict(X_test[['intercept','TV','radio']], results_both)
print('mse test:',mse(y_test, predictions_both_test))

In [ ]:
# Plot residual error for test set

ax = subplots(figsize=(8,8))[1]
ax.scatter(predictions_both_test, y_test-predictions_both_test)
ax.set_xlabel('Fitted value')
ax.set_ylabel('Residual')
ax.axhline(0, c='k', ls='--');

## An example of overfitting via too many predictors

In [ ]:
# What were the correlations among newspaper, TV, and radio?

Train[['TV','radio','newspaper']].corr()

In [ ]:
# Try building a model where TV, radio, and newspaper all predict sales

model_three = sm.OLS(y_train, X_train[['intercept','TV', 'radio', 'newspaper']])
results_three = model_three.fit()
summarize(results_three)

In [ ]:
# R^2

print('R^2 on train:',results_three.rsquared)

In [ ]:
# MSE for "all-three" model

predictions_three_train = predict(X_train[['intercept','TV','radio','newspaper']], results_three)
print('mse train:',mse(y_train, predictions_three_train))
predictions_three_test = predict(X_test[['intercept','TV','radio','newspaper']], results_three)
print('mse test:',mse(y_test, predictions_three_test))

In [ ]:
# For comparison, here were the MSEs for the model with just TV and radio

print('mse train:',mse(y_train, predictions_both_train))
print('mse test:',mse(y_test, predictions_both_test))

### An Example with Interactions

In [ ]:
# We'll create a model that uses TV, radio, and the interaction of the two to predict sales.
# First we need to create the interaction variable and add it to all of the sets.

X_train['TV_radio_int'] = X_train['TV'] * X_train['radio']
X_test['TV_radio_int'] = X_test['TV'] * X_test['radio']
Train['TV_radio_int'] = Train['TV'] * Train['radio']
Test['TV_radio_int'] = Test['TV'] * Test['radio']

In [ ]:
# Might as well look at a correlation matrix with the interaction included

X_train[['TV', 'radio', 'TV_radio_int']].corr()

In [ ]:
# Build a linear model with an interaction term

model_int = sm.OLS(y_train, X_train[['intercept','TV', 'radio', 'TV_radio_int']])
results_int = model_int.fit()
summarize(results_int)

In [ ]:
# R^2 for interaction model

print('R^2 on train:',results_int.rsquared)

In [ ]:
# MSE for interaction model

predictions_int_train = predict(X_train[['intercept','TV','radio','TV_radio_int']], results_int)
print('mse train:',mse(y_train, predictions_int_train))
predictions_int_test = predict(X_test[['intercept','TV','radio','TV_radio_int']], results_int)
print('mse test:',mse(y_test, predictions_int_test))

In [ ]:
# Plot residual error for test set

ax = subplots(figsize=(8,8))[1]
ax.scatter(predictions_int_test, y_test-predictions_int_test)
ax.set_xlabel('Fitted value')
ax.set_ylabel('Residual')
ax.axhline(0, c='k', ls='--');

### One last example: categorical variables

In [ ]:
# Since this particular set doesn't have any categorical variables in it, let's create one artificially
# Let's create a variable that uses the TV data but only takes on values "high" or "low"

X_train['TV_cat'] = pd.Series(np.zeros(X_train.shape[0]))
X_train.loc[X_train['TV']<=30, 'TV_cat'] = "low"
X_train.loc[X_train['TV']>30, 'TV_cat'] = "high"

In [ ]:
# Quick spot check...

X_train.head()

In [ ]:
# Build a linear model with a categorical variable

model_cat = sm.OLS(y_train, X_train[['intercept','radio', 'TV_cat']])
results_cat = model_cat.fit()
summarize(results_cat)

In [ ]:
# We'll need to create a numeric variable to represent each category

X_train['TV_low'] = pd.Series(np.zeros(X_train.shape[0]))
X_train.loc[X_train['TV']<=30, 'TV_low'] = 1
X_train.loc[X_train['TV']>30, 'TV_low'] = 0

X_train['TV_high'] = pd.Series(np.zeros(X_train.shape[0]))
X_train.loc[X_train['TV']<=30, 'TV_high'] = 0
X_train.loc[X_train['TV']>30, 'TV_high'] = 1

In [ ]:
X_train.head()

In [ ]:
# Look at correlations 

X_train[['radio', 'TV_high', 'TV_low']].corr()

In [ ]:
# Fit the model with just the 'TV_low' indicator

model_cat = sm.OLS(y_train, X_train[['intercept','radio','TV_low']])
results_cat = model_cat.fit()
summarize(results_cat)

In [ ]:
# R^2

print('R^2 on train:',results_cat.rsquared)

In [ ]:
# MSE for model w/ categorical variable

predictions_cat_train = predict(X_train[['intercept','radio','TV_low']], results_cat)
print('mse train:',mse(y_train, predictions_cat_train))

In [ ]:
# What if we used the 'TV_high' indicator instead of 'TV_low'?

model_cat2 = sm.OLS(y_train, X_train[['intercept','radio', 'TV_high']])
results_cat2 = model_cat2.fit()
summarize(results_cat2)

In [ ]:
# For comparison, these were the estimates for the 'TV_low' model

summarize(results_cat)

In [ ]:
# R^2

print('train R^2 for TV_high model:',results_cat2.rsquared)
print('train R^2 for TV_low model:',results_cat.rsquared)

In [ ]:
# MSE on train

predictions_cat2_train = predict(X_train[['intercept','radio','TV_high']], results_cat2)
print('mse train for TV_high:',mse(y_train, predictions_cat2_train))
print('mse train for TV_low:',mse(y_train, predictions_cat_train))

## Challenge: Multiple linear regression on auto data

In [ ]:
# Load and review the data

#Auto = pd.read_csv('../../data/Auto.csv')
Auto = pd.read_csv('Auto.csv')
Auto.dtypes

In [ ]:
# Replace the missing horsepowers

Auto.replace({'horsepower':'?'},'104',inplace=True)
Auto['horsepower'] = pd.to_numeric(Auto['horsepower'])

In [ ]:
# Identify some predictors and the target

X = Auto[['horsepower', 'weight', 'year']]
y = Auto['mpg']

In [ ]:
# Create a column for the intercept in the features matrix

X['intercept'] = np.ones(Auto.shape[0])
X

In [ ]:
# Create training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=314,
                                                    test_size=0.25,
                                                    shuffle=True) 
Train = pd.merge_ordered(X_train,y_train,left_on=X_train.index,right_on=y_train.index).drop(columns=['key_0'])
Test = pd.merge_ordered(X_test,y_test,left_on=X_test.index,right_on=y_test.index).drop(columns=['key_0'])

In [ ]:
# Build a linear model where horsepower predicts mpg
# Take note of the p-values

model = sm.OLS(y_train, X_train)
results = model.fit()
summarize(results)

In [ ]:
# We can also assess the overall fit of the model

print('R^2 on train:',results.rsquared)

In [ ]:
# Let's compute the MSE on the training and test sets

predictions_train = predict(X_train, results)
print('mse train:',mse(y_train, predictions_train))
predictions_test = predict(X_test, results)
print('mse test:',mse(y_test, predictions_test))